In [17]:
import sqlite3
import pandas as pd
from difflib import SequenceMatcher
import itertools
import numpy as np
import os
from typing import Generator

os.chdir(os.getcwd().removesuffix('py_scripts'))

In [18]:
conn = sqlite3.connect("database/trades.db") 
cursor = conn.cursor()

# --- Step 2: (a) Load the trades.sql file into the database ---
with open("sql_scripts/trades.sql", "r") as f:
    sql_script = f.read()
cursor.executescript(sql_script)
conn.commit()

# --- Step 3: (b) Read the customer table into a Pandas DataFrame ---
df = pd.read_sql_query("SELECT * FROM customer", conn)
print("Customer table:")
df.head()

Customer table:


,customer_id,company_name,sector_code,country,street,house_number,zip_code,city
0,1,TechRetail BV,S03,NL,Keizersgracht,221,1016AP,Amsterdam
1,2,FinBank AG,S01,DE,Unter den Linden,45,10117,Berlin
2,3,MedTech SA,S06,FR,Rue Lafayette,12,75009,Paris
3,4,EduGlobal BV,S07,NL,Damrak,50,1012AP,Amsterdam
4,5,LogiTrans GmbH,S05,DE,Friedrichstrasse,60,10117,Berlin


In [19]:

# --- Step 4: (c) Define similarity functions ---
def jaro(s, t):
    '''Computes the Jaro similarity between two strings.
    The score ranges from 0 (low) to 1 (high),
    based on the number and order of matching characters
    and the number of transpositions.'''

    if not (isinstance(s,str) or isinstance(t,str)):
        raise ValueError('At least one of the inputs is not a string')
    
    s_len = len(s)
    t_len = len(t)
 
    if s_len == 0 and t_len == 0:
        return 1
 
    match_distance = (max(s_len, t_len) // 2) - 1
 
    s_matches = [False] * s_len
    t_matches = [False] * t_len
 
    matches = 0
    transpositions = 0
 
    for i in range(s_len):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, t_len)
 
        for j in range(start, end):
            if t_matches[j]:
                continue
            if s[i] != t[j]:
                continue
            s_matches[i] = True
            t_matches[j] = True
            matches += 1
            break
 
    if matches == 0:
        return 0
 
    k = 0
    for i in range(s_len):
        if not s_matches[i]:
            continue
        while not t_matches[k]:
            k += 1
        if s[i] != t[k]:
            transpositions += 1
        k += 1
 
    return ((matches / s_len) +
            (matches / t_len) +
            ((matches - transpositions / 2) / matches)) / 3

def jaro_winkler(s, t, p = 0.1):
    '''Computes the Jaro–Winkler similarity between two strings.
    Extends Jaro by giving higher scores to strings that share
    a common prefix, which makes it well-suited for names.'''
    
    jaro_sim = jaro(s, t)
    ls = len(s)
    lt = len(t)
    min_l = min(ls, lt)

    l = 0
    for i in range(min_l):
        if s[i] == t[i]:
            l += 1
        else:
            break
    L = min(l, 4)

    return (jaro_sim + ( L * p * (1 - jaro_sim) ))


def ldist_mod(str1: str, str2: str, cost: int = 2) -> int:
    '''Levenshtein distance between two strings. The distance is the
    minimum number of insertions, deletions, or substitutions
    required to transform one string into the other. The substitution
    cost can be adjusted with the cost parameter.'''

    if not (isinstance(str1,str) or isinstance(str2,str)):
        raise ValueError('At least one of the inputs is not a string')
        
    l1, l2 = len(str1), len(str2)
    mtx = np.zeros((l1+1, l2+1), dtype=int)

    for i in range(l1+1):
        mtx[i][0] = i
    for j in range(l2+1):
        mtx[0][j] = j

    for idx1, c1 in enumerate(str1, start=1):
        for idx2, c2 in enumerate(str2, start=1):
            deli = mtx[idx1-1][idx2] + 1
            inse = mtx[idx1][idx2-1] + 1
            subs = mtx[idx1-1][idx2-1] + (cost if c1 != c2 else 0)
            mtx[idx1][idx2] = min(deli, inse, subs)

    return mtx[l1][l2]


# Alternative Methodology
def ratcliff_obershelp(r1, r2):
    """
    Simple similarity: average SequenceMatcher ratios
    across string-type fields only. -> Ratcliff/Obershelp
    """
    sims = []
    for col in df.columns:
        v1, v2 = str(r1[col]), str(r2[col])
        sims.append(SequenceMatcher(None, v1, v2).ratio())
    return sum(sims) / len(sims)


In [ ]:
# assign weights to fields
def get_weights(df:pd.DataFrame) -> dict:
    'Assign equally weights to columns'
    temp = df.select_dtypes(include=['object', 'string']).columns.tolist()
    return {i:1/len(temp) for i in temp}

def row_similarity(r1:str, r2:str, method:str='jaro_winkler', weights:dict=None) -> Generator:
    sims = []
    for col in weights.keys():
        v1, v2 = str(r1[col]), str(r2[col])
        if method == 'jaro_winkler':
            sims.append(jaro_winkler(v1, v2))
        elif method == 'ratcliff':
            sims.append(SequenceMatcher(None, v1, v2).ratio())
        else:
            raise ValueError(f"Not implemented yet or Unknown method: {method}")
    # weighted average
    return sum(s * weights[col] for s, col in zip(sims, weights.keys()))

def compare_all_pairs(df:pd.DataFrame, weights:dict, method:str='jaro_winkler'):
    results = []
    n = len(df)
    for i in range(n):
        for j in range(i+1, n):
            sim = row_similarity(df.iloc[i], df.iloc[j], method, weights)
            results.append({'row1': i, 'row2': j, 'similarity': sim})
    return pd.DataFrame(results)

In [39]:
weights = get_weights(df)
df_jw = compare_all_pairs(df,weights,method='jaro_winkler')
df_jw[df_jw.similarity>0.7]

,row1,row2,similarity
6,0,7,0.766546
33,3,7,0.735714
65,10,11,0.901020


In [22]:

# --- Step 5: (c) Compare all pairs of customers ---
ro_similar_pairs = []
for i, j in itertools.combinations(df.index, 2):
    sim = ratcliff_obershelp(df.loc[i], df.loc[j])
    if sim > 0.7:
        ro_similar_pairs.append((i, j, sim))

ro_similar_pairs

[(10, 11, 0.7963235294117648)]

In [23]:

# Report results
print("\nCustomers with similarity > 0.7:\n")

for i, j, sim in ro_similar_pairs:
    print(f"Customer {i} and {j} → similarity {sim:.2f}")
    print(df.iloc[[i,j],:],'\n')



Customers with similarity > 0.7:

Customer 10 and 11 → similarity 0.80
    customer_id company_name sector_code country    street house_number  \
10           11   Intesa SPA         S01      IT  Via Roma          10A   
11           12      BPM SPA         S01      IT  Via Roma           13   

   zip_code   city  
10     6200  Milan  
11     6200  Milan   

